In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
input_path = r"D:\桌面\黄金期货历史数据1.csv"
output_path = r"D:\桌面\黄金期货历史数据2.csv"

df = pd.read_csv(input_path)
df = df.iloc[::-1].reset_index(drop=True)
df = df.rename(columns={
    '日期': 'date',
    '交易量': 'volumn'
})

numeric_columns = ['收盘', '开盘', '高', '低']
for col in numeric_columns:
    
    df[col] = df[col].astype(str).str.replace(',', '').astype(float)

df['ave'] = (df['高'] + df['低']) / 2

#处理“volumn”列
def convert_volumn(vol):
    if isinstance(vol, str):
        vol = vol.strip()  # 去除前后空格
        if vol.endswith('K') or vol.endswith('k'):
            return float(vol[:-1]) 
        elif vol.endswith('M') or vol.endswith('m'):
            return float(vol[:-1]) * 1e3
        else:
            # 如果没有单位，尝试直接转换
            return float(vol)
    elif pd.isna(vol):
        return np.nan
    else:
        return float(vol)

df['volumn'] = df['volumn'].apply(convert_volumn)

# 转换“date”列为日期格式
df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d')
df = df.sort_values('date').set_index('date')

# 删除所有2月29日的数据
df = df[~((df.index.month == 2) & (df.index.day == 29))]

full_date_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')
full_date_range = full_date_range[~((full_date_range.month == 2) & (full_date_range.day == 29))]

# 填补缺失数据
df['volumn'] = df['volumn'].interpolate(method='linear')
df['ave'] = df['ave'].interpolate(method='linear')

df = df.reset_index().rename(columns={'index': 'date'})
df['date'] = df['date'].dt.strftime('%Y/%m/%d')

# 创建新的DataFrame
new_df = df[['date', 'volumn', 'ave']]
new_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print("数据处理完成，已保存到:", output_path)


数据处理完成，已保存到: D:\桌面\黄金期货历史数据2.csv
